*Note: all those exercises should be done using python with pyccel openmp*

## Exercise 1 Hello World

1. Write an OpenMP program displaying the number of threads used for the execution and the rank of each of the threads.
3. Test the programs obtained with different numbers of threads for the parallel program.

**Output Example**
```shell
>> Python hello.py
Hello from the rank 2 thread
Hello from the rank 0 thread
Hello from the rank 3 thread
Hello from the rank 1 thread
Hello from the rank 4 thread
Parallel execution of hello_world with 4 threads
```
*Note that the output order maybe different*

In [31]:
import numpy as np
import time
from pyccel.epyccel import epyccel
from pyccel.decorators import types

In [32]:
%%file hello.py
from pyccel.stdlib.internal.openmp import omp_set_num_threads, omp_set_dynamic
from pyccel.decorators import types
from pyccel.stdlib.internal.openmp import omp_set_num_threads, omp_get_num_threads, omp_get_thread_num

@types('int')
def get_num_threads(n):
    #omp_set_dynamic(0)
    omp_set_num_threads(int(n))
    #$ omp parallel
    print("Hello from this$ omp end parallele rank ", int(omp_get_thread_num())," thread")
    result = omp_get_num_threads()
    #$ omp end parallel
    
    return result

if __name__ == "__main__" :
    x = get_num_threads(7)
    print("parallel execution of hello_world with ",x , "threads")

Writing hello.py


In [33]:
# enter command for compile and run the program
!pyccel hello.py --language=c --openmp

In [34]:
!./hello

Hello from this$ omp end parallele rank  0  thread
Hello from this$ omp end parallele rank  3  thread
Hello from this$ omp end parallele rank  4  thread
Hello from this$ omp end parallele rank  5  thread
Hello from this$ omp end parallele rank  6  thread
Hello from this$ omp end parallele rank  2  thread
Hello from this$ omp end parallele rank  1  thread
parallel execution of hello_world with  7 threads


## Exercise 2 Matrix product

Considering the following code for matrix product:

1. Using pyccel, epyccelize the `matrix_prod` function, and time the execution of epyccelized function.
2. Insert the appropriate OpenMP directives and analyse the code performance.
3. Test the loop iteration repartition modes (`STATIC`, `DYNAMIC`, `GUIDED`) and vary the chunk sizes.
**Example output**

```bash
   Value of M and N           : 20 20 
    Temps elapsed              : 0.0009012222290039062 
    Temps CPU                  : 0.0 
    Partiel results            : 2790.0 2690.0 ... -3610.0 -4350.0 

 Execution of Matrix production in parallele with                     8 threads
```

In [35]:
def matrix_prod(A, B, C, N, M):
    
    for i in range( M ):
        for j in range( N ):
            A[ i, j ] = (i + 1) + (j + 1)

    
    for i in range( N ):
        for j in range( M ):
            B[ i, j ] = (i + 1) - (j + 1)

    
    for i in range( M ):
        for j in range( M ):
            C[ i, j ] = 0
    # ...

    # Matrix Production
    
    for i in range( M ):
        for j in range( M ):
            for k in range( N ):
                C[ i, j ] += A[ i, k ] * B[ k, j ]




if __name__ == "__main__":
    import numpy as np
    import time
    import math

    M = 20
    N = 20

    
    A = np.empty((M, N), dtype=np.double)
    B = np.empty((N, M), dtype=np.double)
    C = np.empty((M, M), dtype=np.double)

#  time of the matrix
%time matrix_prod(A,B,C,N,M)

CPU times: user 19.3 ms, sys: 0 ns, total: 19.3 ms
Wall time: 36.5 ms


## using PYCCEL

In [36]:
def matrix_prod_pyccel(A : 'float[:,:]', B : 'float[:,:]', C:'float[:,:]', N : 'int', M :'int'):
    
    for i in range( M ):
        for j in range( N ):
            A[ i, j ] = (i + 1) + (j + 1)

    
    for i in range( N ):
        for j in range( M ):
            B[ i, j ] = (i + 1) - (j + 1)

    
    for i in range( M ):
        for j in range( M ):
            C[ i, j ] = 0
    # ...

    # Matrix Production
    
    for i in range( M ):
        for j in range( M ):
            for k in range( N ):
                C[ i, j ] += A[ i, k ] * B[ k, j ]

In [37]:
import numpy as np
import time
import math

M = 20
N = 20

# Declare Matrices
A = np.empty((M, N), dtype=np.double)
B = np.empty((N, M), dtype=np.double)
C = np.empty((M, M), dtype=np.double) 

In [38]:
#  run and time the matrix_prod function
matrix_prod_pyccel_form = epyccel(matrix_prod_pyccel)
%time matrix_prod_pyccel_form(A,B,C,N,M)   

CPU times: user 0 ns, sys: 135 µs, total: 135 µs
Wall time: 6.44 ms


## Using OpenMP 

In [39]:
%%file matrice_prod.py
from pyccel.stdlib.internal.openmp import omp_set_num_threads, omp_set_dynamic
from pyccel.decorators import types
from pyccel.stdlib.internal.openmp import omp_set_num_threads, omp_get_num_threads, omp_get_thread_num
import numpy as np

@types('int','int')
def pro_mat_collaps(M,N):
    
    
    print ("N = ", N,"  M = ", M,"\n" )
    
    A = np.empty((M, N))
    B = np.empty((N, M))
    C = np.empty((M, M))
    
    
    #$ omp parallel
    #$ omp for collapse(2)
    for i in range(M):
        for j in range(N):
            A[ i, j ] = (i + 1) + (j + 1)
    #$ omp end parallel
    

    #$ omp parallel
    #$ omp for collapse(2)
    for i in range(N):
        for j in range(M):
            B[ i, j ] = (i + 1) - (j + 1)
    #$ omp end parallel

    #$ omp parallel
    #$ omp for collapse(2)
    for i in range( M ):
        for j in range( M ):
            C[ i, j ] = 0
    #$ omp end parallel

    
    #$ omp parallel
    #$ omp for collapse (2)
    for i in range( M ):
        for j in range( M ):
            for k in range( N ):
                C[ i, j ] += A[ i, k ] * B[ k, j ]
    result = omp_get_num_threads()
    #$ omp end parallel
   
    return result

if __name__ == "__main__" :

    M = 20
    N = 20
    # Declare Matrices

    x = pro_mat_collaps(M,N)
    print("Execution of Matrix production in parallele with  ",x , "threads\n\n")

Writing matrice_prod.py


In [41]:
# enter command for compile and run the program
!pyccel matrice_prod.py --language=c --openmp

In [42]:
%time !export OMP_NUM_THREADS=8; ./matrice_prod

N =  20   M =  20 

Execution of Matrix production in parallele with   8 threads


CPU times: user 5.34 ms, sys: 4.62 ms, total: 9.95 ms
Wall time: 147 ms


## Exercise 3 Jacobi method

Considering the following code for a general linear system solver:

$$ A \times x = b $$

using the Jacobi iterative method.
In this exercice, you must solve the system in parallel
   
**Example Output**
    
```bash
System size    : 20 
 Iterations     : 21 
 Stand          : 2.9100049961372497e-08 
 Elapsed Time   : 7.447303771972656 
 CPU time       : 0.921875 
```

In [43]:

def jacobi(a:'float [:,:]',b:'float [:]', x:'float [:]', n:'int', eps:'float'):

#     print("Execution of Jacobi in parallel with", nb_tasks, "threads")
    # Jacobi method resolution
    import numpy as np
    import math
    x_courant = np.empty(n, dtype=np.double)
    iteration = 0

    #$ omp parallel
    while(1):
        
        iteration += 1
        
        for i in range(n):
            x_courant[i] = 0
            for j in range(i):
                x_courant[i] += a[ j, i ] * x[j]

            for j in range(i + 1, n):
                x_courant[i] += a[ j, i ] * x[j]

            x_courant[i] = (b[i] - x_courant[i]) / a[ i, i ]

        # Convergence test
        
        absmax = 0.
        for i in range(n):
            curr = abs(x[i] - x_courant[i])
            if curr > absmax:
                absmax = curr
        stand = absmax / n
        if stand <= eps or iteration > n:
            break

        # copy x_courant into x
        for i in range(n):
            x [i] = x_courant[i]
            
    #$ omp end parallel    
            
    return stand, iteration


if __name__ == "__main__":

    import numpy as np
    import time, random, math, sys

    # Matrix default dimension
    n = 20
    diag = 20

    # Initialize of the matrices
    np.random.seed(421)  # for reproductibles results
    a = np.random.uniform(0, 1, size=(n, n))
    b = np.random.uniform(0, 1, size=n)

    # We strengthen the main diagonal of the matrix
    for i in range(n):
        a[ i, i ] += diag

    # Initial solution
    x = np.ones(n, dtype=np.double)

    # start CPU timing
    cpu_0 = time.process_time()

    # start Wall timing
    elapsed_0 = time.time()

    # using pyccel
    from pyccel.epyccel import epyccel
    pyccel_jacobi = epyccel(jacobi, language='c', accelerators=['openmp'])

    eps = sys.float_info.epsilon

    # -------------------- computing ------------------
    stand, iteration = pyccel_jacobi(a, b, x, n, eps)
    # ------------------ End computing ----------------

    # CPU time
    cpu_1 = time.process_time()
    cpu = cpu_1 - cpu_0

    # Wall timing
    elapsed_1 = time.time()
    elapsed = elapsed_1 - elapsed_0

    print(
        "System size    :", n,          '\n',
        "Iterations     :", iteration,  '\n',
        "Stand          :", stand,      '\n',
        "Elapsed Time   :", elapsed,    '\n',
        "CPU time       :", cpu,        '\n'
    )

System size    : 20 
 Iterations     : 21 
 Stand          : 2.9100049961372497e-08 
 Elapsed Time   : 1.8009474277496338 
 CPU time       : 0.09848454900000014 



## Exercise 4 Calculation of π

The aim of this exercise is to calculate π by numerical integration knowing that Considering the following code for matrix product: $\int_{0}^{1} \frac{4}{1 + x^2} = {\pi} $

The following program is for calculating the value of
π by the rectangle method (mid-point). Let $f(x) = \frac{4}{1 + x^2} $
2 be the function to integrate, N and $ h = \frac{1}{N} $ (respectively) the number of points, and the discretization width on the integration
interval $[0, 1]$.

Analyse the performance of the codes, then optimise  your code using OpenMP directives.

In [55]:
%%file pi.py
from pyccel.decorators import types

@types('int')
def pi(n):

    h = float(1/n)
    for k in range(100):
        Pi_calcul = 0.
        for i in range(n):
            x = h * (i + 0.5)
            Pi_calcul += 4.0 / (1. + x * x)


        Pi_calcul = h * Pi_calcul
    return Pi_calcul

Writing pi.py


In [56]:
from pi import *

In [57]:
pi(2)

3.1623529411764704

In [61]:
import time, math

if __name__ == '__main__':
    #                  __
    #  But : calcul de || par la methode des rectangles (point milieu).
    #
    #                   / 1
    #                  |       4            __
    #                  |   ----------  dx = ||
    #                  |    1 + x**2
    #                 / 0


    n = 300

    # Length of the integration interval
    h = 1.0 / n

    from pyccel.epyccel import epyccel
    import pi
    
    pyccel_pi = epyccel(pi, language='c', accelerators=['openmp'])

    # start timing
    tcpu_0 = time.process_time() 
    t_elapsed_0 = time.time()    
     # -------------------- computing ------------------

    Pi_calc = pyccel_pi(n)

    # ------------------ End computing ----------------

    # CPU time spent
    tcpu_1 = time.process_time()
    tcpu = tcpu_1 - tcpu_0

    # Wall time spent
    t_elapsed_1 = time.time()
    t_elapsed = t_elapsed_1 - t_elapsed_0

    # deviation between the estimated value and the calculated value of Pi
    Pi_estime = math.acos(-1)
    deviat = math.fabs(Pi_estime -  Pi_calc)

    # Print result
    print(
      "   Number of intervals        :", n,         "\n",
      "   | Pi_estime - Pi_calcule | :", deviat,    "\n",
      "   Temps elapsed              :", t_elapsed, "\n",
      "   Temps CPU                  :", tcpu,      "\n",
    )

TypeError: 'module' object is not callable

## Exercise 5 Reduction of an array

The given program contained in the `reduction_tab.py` file is extracted from a chemistry code. It reduces a three-dimensional array into a vector. The aim of this exercise is to parallelize this calculation kernel without changing the loop order in the provided code (i.e. k,j,i)

1. Analyse the data-sharing attributes of the variables and adapt the source code so that the K outermost loop is parallelized.
2. Compare the performance obtained by using the thread/core binding default execution on Ada and by using scatter binding. Suggest an explanation for the poor performance of the latter.
3. Optimise the source code for the scatter mode with taking into account the memory affinity. Why does this third series of executions give the best performance 

In [65]:
%%file reduction_tab.py

from pyccel.stdlib.internal.openmp import omp_get_thread_num
from pyccel.decorators import types


def initialize_table(tab, nmolec, n, nmol):
    # Initialisation du tableau
    # First-touch pour garantir un fonctionnement optimal sur les systemes NUMA
    for k in range(nmolec):
        for j in range(n):
            for i in range(nmol):
                tab[ i,j,k ] = i + j + k


def reduction(tab:'float [:,:]', tab1 : 'float [:]', tab2 : 'float [:]', nmolec :'int', nmol:'int', n:'int'):
    for k in range(nmolec):
        tab1[:nmol] = 0
        for j in range(n):
            for i in range(nmol):
                tab1[i] = tab1[i] + tab[i,j,k]
        tab2[:nmol] = tab2[:nmol] + 2 * tab1[:nmol]
        


Writing reduction_tab1.py


In [67]:
from os import error

if __name__ == '__main__':
    import numpy as np
    import time

    # default dimension
    nmolec = 100
    nmol   = 100
    n      = 10

    tab     = np.empty((nmol, n, nmolec), dtype=np.double)
    tab1    = np.empty((nmol), dtype=np.double)
    tab2    = np.zeros((nmol), dtype=np.double)

    # using pyccel
    from pyccel.epyccel import epyccel
    import  reduction_tab
    
    reduction_puccel = epyccel(reduction, language='c', accelerators=['openmp'])
    

    # initialize tab 
    initialize_table(tab, nmol, n, nmolec)

    # start timing
    tcpu_0 = time.process_time() 
    t_elapsed_0 = time.time()    

    # -------------------- computing ------------------
    reduction.reduction(tab, tab1, tab2, nmolec, nmol, n)
    # ------------------ End computing ----------------

    # CPU time spent
    tcpu_1 = time.process_time()
    tcpu = tcpu_1 - tcpu_0

    # Wall time spent
    t_elapsed_1 = time.time()
    t_elapsed = t_elapsed_1 - t_elapsed_0

    # verification of results
    tab2c = np.zeros((nmol), dtype=np.double)
    for k in range(nmolec):
        tab1c = np.zeros((nmol), dtype=np.double)
        for j in range(n):
            for i in range(nmol):
                tab1c[i] = tab1c[i] + tab[i, j, k]
        tab2c [:] = tab2c[:nmol] + 2 * tab1c[:]
    err = np.max(np.abs(tab2c - tab2) / np.abs(tab2c))
    print(
        "Temps elapsed  :", t_elapsed   , "\n",
        "Temps CPU      :", tcpu        , "\n",
        "Erreur relative:", err         , "\n"
    )

NameError: name 'reduction' is not defined

# Exercise 6 Matrix product by the Strassen algorithm

Considering the following code that calculates the matrix product :

$$C = A \times B$$

by using Strassen’s recursive algorithm.

In this exercise, you must :
1. Analyse and parallelize the code by using OpenMP tasks.
2. Measure the code performance and plot the speedup curves obtained.

In [69]:
import time, math
import numpy as np
from pyccel.epyccel import epyccel
from pyccel.decorators import types

def threads_num():
#     complete this function for getting threads num
    res = opm_get_thread_num()
    return res



def strassen_multiply(A, B, C, n):

    
    import numpy as np
    if n & 1 != 0  or n < 128:
        C[:] = np.matmul(A, B)
    else:
        n2 = n // 2
        A11 = A[ 0:n2, 0:n2 ]
        A21 = A[ n2:n, 0:n2 ]
        A12 = A[ 0:n2, n2:n ]
        A22 = A[ n2:n, n2:n ]
        B11 = B[ 0:n2, 0:n2 ]
        B21 = B[ n2:n, 0:n2 ]
        B12 = B[ 0:n2, n2:n ]
        B22 = B[ n2:n, n2:n ]

        Q1 = np.empty((n2, n2), dtype=np.double)
        Q2 = np.empty((n2, n2), dtype=np.double)
        Q3 = np.empty((n2, n2), dtype=np.double)
        Q4 = np.empty((n2, n2), dtype=np.double)
        Q5 = np.empty((n2, n2), dtype=np.double)
        Q6 = np.empty((n2, n2), dtype=np.double)
        Q7 = np.empty((n2, n2), dtype=np.double)
        
        strassen_multiply(A11+A22, B11+B22, Q1, n2)
        strassen_multiply(A21+A22, B11, Q2, n2)
        strassen_multiply(A11, B12-B22, Q3, n2)
        strassen_multiply(A22, -B11+B21, Q4, n2) 
        strassen_multiply(A11+A12, B22, Q5, n2)
        strassen_multiply(-A11+A21, B11+B12, Q6, n2)
        strassen_multiply(A12-A22, B21+B22, Q7, n2)
        C[   :n2,   :n2 ] = Q1+Q4-Q5+Q7
        C[ n2: n,   :n2 ] = Q2+Q4
        C[   :n2, n2:n  ] = Q3+Q5
        C[ n2:n , n2:n  ] = Q1+Q3-Q2+Q6


def calcul_erreur(n, A, B, C):
    D = np.matmul(A, B) 
    error = 0.0
    for i in range(n):
        for j in range(n):
            e = C[ i, j ] - D[ i, j ]
            error += e * e

    error = math.sqrt(error) / n
    return error

if __name__ == '__main__':
    n = 256

    # Initialize Matrices
    np.random.seed(421)
    A = np.random.uniform(0, 1, (n, n))
    B = np.random.uniform(0, 1, (n, n))
    C = np.empty((n,n), dtype=np.double)

    from pyccel.epyccel import epyccel
    strassen_pyccel = epyccel(strassen_multiply, accelerators=['openmp'])
    num_thread_pyccel = epyccel(threads_num, accelerators=['openmp'])
    print("Execution of Strassen in parallele with", num_threads(), "threads")
    
    
    # Start timing
    tcpu_0 = time.process_time() # CPU time
    t_elapsed_0 = time.time()    # Wall time
    # -------------------- computing ------------------
    # Calcul C=A*B par la methode recursive de Strassen
    strassen_pyccel(A, B, C, n)
    # ------------------ end computing ----------------
    # CPU time spent
    tcpu_1 = time.process_time()
    tcpu = tcpu_1 - tcpu_0

    # Wall time spent
    t_elapsed_1 = time.time()
    t_elapsed = t_elapsed_1 - t_elapsed_0
    # Print result
    print(
      "   Erreur        :", calcul_erreur(n, A, B, C),  "\n",
	  "   Temps elapsed :", t_elapsed,                  "\n",
	  "   Temps CPU     :", tcpu,                       "\n",
    )



ERROR at annotation (semantic) stage
pyccel:
 |error [semantic]: mod_yvpdxl36zajk_yvpdxl36zajk.py [1]| Function is missing a type annotation (strassen_multiply)
 |fatal [semantic]: mod_yvpdxl36zajk_yvpdxl36zajk.py [5,7]| Undefined variable (n)



PyccelSemanticError: Semantic step failed